In [1]:
import torch

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [-2.7, -1.5]
])

y_train = torch.tensor(
    [0, 0, 0, 1, 1]
)

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])

y_test = torch.tensor([0, 1])

In [2]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):

  def __init__(self, X, y):
    self.features = X
    self.labels = y

  def __getitem__(self, index):
    one_x = self.features[index]
    one_y = self.labels[index]

    return one_x, one_y

  def __len__(self):
    return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [6]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last = True # If the size of the last batch is different it can cause error in convergence, so we just drop the last in every epoch
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [7]:
enumerate(train_loader)

In [8]:
for idx, (x, y) in enumerate(train_loader):
  print(idx)
  print(f"Batch index: {idx+1}:", x, y)

0
Batch index: 1: tensor([[-2.7000, -1.5000],
        [-0.9000,  2.9000]]) tensor([1, 0])
1
Batch index: 2: tensor([[ 2.3000, -1.1000],
        [-1.2000,  3.1000]]) tensor([1, 0])


## Training a Neural Network

In [20]:
import torch
import torch.nn.functional as F

class NeuralNetwork(torch.nn.Module):

  def __init__(self, input_nums, output_nums):
    super().__init__()
    self.layers = torch.nn.Sequential(
        torch.nn.Linear(input_nums, 10),
        torch.nn.ReLU(),

        torch.nn.Linear(10, 7),
        torch.nn.ReLU(),

        torch.nn.Linear(7, 5),
        torch.nn.ReLU(),

        torch.nn.Linear(5, output_nums)
    )

  def forward(self, x):
    return self.layers(x)

model = NeuralNetwork(2, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 10

for epoch in range(num_epochs):

  model.train()

  for batch_idx, (features, label) in enumerate(train_loader):

    # Since we are training here we will need the auto_grad
    logits = model(features)

    loss = F.cross_entropy(logits, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Logging
    print(f"Epoch: {epoch+1}, Batch: {batch_idx+1}, Train Loss = {loss:.2f}")

Epoch: 1, Batch: 1, Train Loss = 0.73
Epoch: 1, Batch: 2, Train Loss = 0.72
Epoch: 2, Batch: 1, Train Loss = 0.89
Epoch: 2, Batch: 2, Train Loss = 0.86
Epoch: 3, Batch: 1, Train Loss = 0.54
Epoch: 3, Batch: 2, Train Loss = 0.74
Epoch: 4, Batch: 1, Train Loss = 0.67
Epoch: 4, Batch: 2, Train Loss = 0.73
Epoch: 5, Batch: 1, Train Loss = 0.54
Epoch: 5, Batch: 2, Train Loss = 1.05
Epoch: 6, Batch: 1, Train Loss = 0.50
Epoch: 6, Batch: 2, Train Loss = 0.62
Epoch: 7, Batch: 1, Train Loss = 0.44
Epoch: 7, Batch: 2, Train Loss = 0.58
Epoch: 8, Batch: 1, Train Loss = 0.27
Epoch: 8, Batch: 2, Train Loss = 0.29
Epoch: 9, Batch: 1, Train Loss = 0.01
Epoch: 9, Batch: 2, Train Loss = 0.31
Epoch: 10, Batch: 1, Train Loss = 0.12
Epoch: 10, Batch: 2, Train Loss = 0.10


In [21]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad == True )

print(num_params)

159


In [24]:
# Model evaluation
model.eval()

with torch.no_grad():
  outputs = model(X_train)

print(outputs)

torch.set_printoptions(sci_mode=False)
probs = torch.softmax(outputs, dim=1)

print(probs)

predictions = torch.argmax(probs, dim=1)


print(predictions)

tensor([[ 0.6826, -0.9908],
        [ 0.6760, -0.9936],
        [ 0.6671, -0.9974],
        [-2.9177,  1.9537],
        [-3.2001,  2.1970]])
tensor([[0.8420, 0.1580],
        [0.8415, 0.1585],
        [0.8408, 0.1592],
        [0.0076, 0.9924],
        [0.0045, 0.9955]])
tensor([0, 0, 0, 1, 1])


In [32]:
def compute_accuracy(model, dataloader):

  model = model.eval()
  correct = 0.0
  total_examples = 0

  for idx, (features, labels) in enumerate(dataloader):

    with torch.no_grad():
      logits = model(features)

    logits = torch.softmax(logits, dim=1)

    predictions = torch.argmax(logits, dim=1)

    compare = labels == predictions

    correct += torch.sum(compare)

    total_examples += len(compare)

  return (correct / total_examples).item()

In [33]:
print(compute_accuracy(model, train_loader))

1.0


In [34]:
print(compute_accuracy(model, test_loader))

1.0


In [35]:
torch.save(model.state_dict(), "model.pth")

In [36]:
model1 = model.load_state_dict(torch.load("model.pth"))

<ipython-input-36-f49d3d3e6e5c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1 = model.load_state_dict(torch.load("model.pth"))
